In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.metrics import brier_score_loss

import xgboost as xgb
from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)

In [36]:
spread_df=[]
years_list = [20,19,18,17,16,15,14,13,12,11,10,'09','08','07','06','05','04','03']
year_int = 2021
for year in years_list:
    #Note: 19 refers to 2020 and so on
    temp_spread_df = pd.read_csv("C:/Users/FLUXNATURE/Desktop/New Kaggle world/Point Spread/ncaabb" + str(year) + ".csv")
    temp_spread_df['Season'] = year_int
    year_last = year
    year_int = year_int-1    
    if year==20:
        spread_df = temp_spread_df
    else:
        spread_df = spread_df.append(temp_spread_df)

In [37]:
spread_df.head(100)

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linesaggm,linemoore,lineopen,linedok,linefox,std,linepugh,linedonc,neutral,linetalis,lineespn,linepir,linepiw,linepib,line7ot,lineer,linedd,linemassey,linedunk,lineround,lineteamrnks,Season,lineashby,lineprophet,lineseven,linepig,linepom,linemass,linerpi,linegreen,lineash,lineargh,linemarkov,linesauce
0,NaN,Abilene Christian,70,East Tennessee St.,47,6.5,-5.46,-7.34,-7.34,-7.34,-7.34,1.46,-1,-9.32,-10.0,4.20,-1.32,-2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.71,NaN,NaN,0.2,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/25/2020,Alabama,81,Jacksonville St.,57,20.5,17.53,16.94,16.94,16.94,16.94,19.64,18,16.72,16.0,2.30,22.82,13.8,0.0,NaN,17.36,NaN,NaN,NaN,NaN,NaN,NaN,16.30,NaN,NaN,20.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/25/2020,Arkansas,142,Miss Valley St.,62,36.5,31.78,30.21,30.21,30.21,30.21,29.45,37,NaN,34.0,4.05,41.43,25.1,0.0,NaN,33.47,NaN,NaN,NaN,NaN,NaN,NaN,31.74,NaN,NaN,33.6,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11/25/2020,Arkansas-Little Rock,71,Prairie View,66,14,7.70,7.31,7.31,7.31,7.31,11.05,11.5,NaN,7.0,2.70,11.15,NaN,1.0,NaN,2.78,NaN,NaN,NaN,NaN,NaN,NaN,4.93,NaN,NaN,10.8,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/25/2020,Boston College,67,Villanova,76,-14,-13.40,-13.53,-13.53,-13.53,-13.53,-14.79,-14,-16.65,-17.0,2.55,-13.88,-10.8,1.0,NaN,-10.34,NaN,NaN,NaN,NaN,NaN,NaN,-8.20,NaN,NaN,-15.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11/26/2020,Towson,68,San Francisco,79,-7.5,-5.29,-1.21,-1.79,-5.90,4.60,-12.47,-8,-6.30,-9.0,4.64,-10.06,-5.9,1.0,NaN,-1.61,NaN,NaN,NaN,-9.0,NaN,NaN,-7.85,NaN,NaN,-6.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,11/26/2020,Utah St.,59,South Dakota St.,83,-2,2.71,-0.02,-0.04,3.24,-2.56,1.89,-2,7.08,5.0,3.21,5.95,5.9,1.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.22,NaN,NaN,-0.8,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,11/26/2020,West Virginia,78,VA Commonwealth,66,10.5,7.40,7.08,6.46,9.84,3.14,15.45,9,2.51,6.0,3.77,9.95,8.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.52,NaN,NaN,9.5,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,11/26/2020,Xavier,51,Bradley,50,10,8.45,7.12,7.61,8.32,5.11,9.05,11,10.53,13.0,3.43,9.21,7.9,0.0,NaN,8.26,NaN,NaN,NaN,7.0,NaN,NaN,1.00,NaN,NaN,14.3,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
spread_df['home'] = spread_df['home'].str.lower()
spread_df['road'] = spread_df['road'].str.lower()

In [39]:
spread_df.head()

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linesaggm,linemoore,lineopen,linedok,linefox,std,linepugh,linedonc,neutral,linetalis,lineespn,linepir,linepiw,linepib,line7ot,lineer,linedd,linemassey,linedunk,lineround,lineteamrnks,Season,lineashby,lineprophet,lineseven,linepig,linepom,linemass,linerpi,linegreen,lineash,lineargh,linemarkov,linesauce
0,NaN,abilene christian,70,east tennessee st.,47,6.5,-5.46,-7.34,-7.34,-7.34,-7.34,1.46,-1,-9.32,-10.0,4.20,-1.32,-2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.71,NaN,NaN,0.2,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/25/2020,alabama,81,jacksonville st.,57,20.5,17.53,16.94,16.94,16.94,16.94,19.64,18,16.72,16.0,2.30,22.82,13.8,0.0,NaN,17.36,NaN,NaN,NaN,NaN,NaN,NaN,16.30,NaN,NaN,20.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/25/2020,arkansas,142,miss valley st.,62,36.5,31.78,30.21,30.21,30.21,30.21,29.45,37,NaN,34.0,4.05,41.43,25.1,0.0,NaN,33.47,NaN,NaN,NaN,NaN,NaN,NaN,31.74,NaN,NaN,33.6,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11/25/2020,arkansas-little rock,71,prairie view,66,14,7.70,7.31,7.31,7.31,7.31,11.05,11.5,NaN,7.0,2.70,11.15,NaN,1.0,NaN,2.78,NaN,NaN,NaN,NaN,NaN,NaN,4.93,NaN,NaN,10.8,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/25/2020,boston college,67,villanova,76,-14,-13.40,-13.53,-13.53,-13.53,-13.53,-14.79,-14,-16.65,-17.0,2.55,-13.88,-10.8,1.0,NaN,-10.34,NaN,NaN,NaN,NaN,NaN,NaN,-8.20,NaN,NaN,-15.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Match Team IDs to data from NCAAM

In [40]:
teams_df = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM_stage_two_two/MDataFiles_Stage2/MTeamSpellings.csv', sep='\,', engine='python')

In [41]:
teams_df.head(3)

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101


In [42]:
teams_df.tail(3)

,TeamNameSpelling,TeamID
1144,youngstown state,1464
1145,youngstown-st,1464
1146,youngstown-state,1464


In [43]:
team_list = ['home','road']
for team in team_list:
    spread_df = pd.merge(spread_df, teams_df, left_on=[team], right_on = ['TeamNameSpelling'], how='left')
    if team=='home':
        spread_df.rename(columns={'TeamID': 'HTeamID'}, inplace=True)
    if team=='road':
        spread_df.rename(columns={'TeamID': 'RTeamID'}, inplace=True)
    spread_df = spread_df.drop(['TeamNameSpelling'], axis=1)

mudifying home and road to team and score

see implemetation below

In [44]:
spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WTeamID'] = spread_df['HTeamID']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LTeamID'] = spread_df['HTeamID']

spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WScore'] = spread_df['hscore']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LScore'] = spread_df['hscore']

In [45]:
spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WTeamID'] = spread_df['RTeamID']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LTeamID'] = spread_df['RTeamID']

spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WScore'] = spread_df['rscore']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LScore'] = spread_df['rscore']

In [46]:
spread_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['WTeamID','LTeamID','WScore','LScore','line'])
spread_df = spread_df[spread_df['WScore'] != "."]
spread_df = spread_df[spread_df['LScore'] != "."]
spread_df = spread_df[spread_df['line'] != "."]

In [47]:
spread_df['line'].value_counts(dropna=False)

3.5      1830
2.5      1828
3.0      1819
5.5      1788
5.0      1770
         ... 
46.0        1
26.5        1
-17.0       1
-37.0       1
-32.5       1
Name: line, Length: 273, dtype: int64

In [48]:
#drop spreads with bad coverage
spread_df = spread_df.drop(['line7ot','lineargh','lineash','lineashby','linedd','linedunk','lineer','linegreen','linemarkov','linemass','linepib','linepig','linepir','linepiw','linepom','lineprophet','linerpi','lineround','linesauce','lineseven','neutral','lineteamrnks','linetalis','lineespn','linemassey','linedonc','linesaggm','std','linepugh','linefox','linedok','lineopen'], axis=1)

In [49]:
spread_df.tail()

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linemoore,Season,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
67700,03/30/2004,iowa st.,81,rutgers,84,2.0,1.22,1.50,1.25,1.74,1.61,2004,1235.0,1353.0,1353.0,1235.0,84,81
67701,04/01/2004,michigan,62,rutgers,55,5.5,2.06,1.51,0.46,2.44,4.47,2004,1276.0,1353.0,1276.0,1353.0,62,55
67702,04/03/2004,duke,78,connecticut,79,-2.0,2.85,2.85,2.24,3.94,1.43,2004,1181.0,1163.0,1163.0,1181.0,79,78
67703,04/03/2004,oklahoma st.,67,georgia tech,67,4.5,2.05,1.99,3.58,0.52,3.08,2004,1329.0,1210.0,NaN,NaN,NaN,NaN
67704,04/05/2004,connecticut,82,georgia tech,73,5.0,1.65,1.12,1.31,0.91,4.57,2004,1163.0,1210.0,1163.0,1210.0,82,73


In [50]:
spread_df.dropna(subset=['line'])

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linemoore,Season,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
0,NaN,abilene christian,70,east tennessee st.,47,6.5,-5.46,-7.34,-7.34,-7.34,1.46,2021,1101.0,1190.0,1101.0,1190.0,70,47
1,11/25/2020,alabama,81,jacksonville st.,57,20.5,17.53,16.94,16.94,16.94,19.64,2021,1104.0,1240.0,1104.0,1240.0,81,57
2,11/25/2020,arkansas,142,miss valley st.,62,36.5,31.78,30.21,30.21,30.21,29.45,2021,1116.0,1290.0,1116.0,1290.0,142,62
3,11/25/2020,arkansas-little rock,71,prairie view,66,14,7.70,7.31,7.31,7.31,11.05,2021,1114.0,1341.0,1114.0,1341.0,71,66
4,11/25/2020,boston college,67,villanova,76,-14,-13.40,-13.53,-13.53,-13.53,-14.79,2021,1130.0,1437.0,1437.0,1130.0,76,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67700,03/30/2004,iowa st.,81,rutgers,84,2.0,1.22,1.50,1.25,1.74,1.61,2004,1235.0,1353.0,1353.0,1235.0,84,81
67701,04/01/2004,michigan,62,rutgers,55,5.5,2.06,1.51,0.46,2.44,4.47,2004,1276.0,1353.0,1276.0,1353.0,62,55
67702,04/03/2004,duke,78,connecticut,79,-2.0,2.85,2.85,2.24,3.94,1.43,2004,1181.0,1163.0,1163.0,1181.0,79,78
67703,04/03/2004,oklahoma st.,67,georgia tech,67,4.5,2.05,1.99,3.58,0.52,3.08,2004,1329.0,1210.0,NaN,NaN,NaN,NaN


Write the data to a csv

In [51]:
spread_df.to_csv('spreads_all.csv', index=False)